# main

In [ ]:
from __future__ import annotations

import json
import os

from tqdm import tqdm

import character_utility as charutil
import config
from ipywidgets_helper import render_images
from kvg import Kvg
from utility import pathstr, char2code

In [ ]:
def test():
    kvg = Kvg.parse("祭", log=True)
    print()
    
    print(json.dumps(kvg.to_dict(), ensure_ascii=False, indent=2))
    print()
    
    images = kvg.draw(image_size=64, padding=4, stroke_width=2)
    
    return render_images([image for _, image in images])


test()

## JSON

In [ ]:
def save_jsons(chars: list[str]) -> None:
    pbar = tqdm(chars)
    for char in pbar:
        kvg = Kvg.parse(char)
        pbar.set_postfix(kvgid=kvg.kvgid, name=kvg.name)
        
        assert kvg.kvgid == kvg.charcode
        
        directory_path = config.output_main_kvg_path(kvg.charcode)
        os.makedirs(directory_path, exist_ok=True)
        
        with open(pathstr(directory_path, f"{kvg.charcode}.json"), "w") as f:
            json.dump(kvg.to_dict(), f)


# save_jsons(charutil.kanjis.all())

## PNG

In [ ]:
def save_pngs(chars: list[str], image_size: int, padding: int, stroke_width: int) -> None:
    pbar = tqdm(chars)
    for char in pbar:
        charcode = char2code(char)
        directory_path = config.output_main_kvg_path(charcode)
        with open(pathstr(directory_path, f"{charcode}.json")) as f:
            kvg = Kvg.from_dict(json.load(f))
        pbar.set_postfix(kvgid=kvg.kvgid, name=kvg.name)
        
        images = kvg.draw(image_size=image_size, padding=padding, stroke_width=stroke_width)
        for kvg, image in images:
            directory_path = config.output_main_kvg_path(kvg.charcode)
            os.makedirs(directory_path, exist_ok=True)
            
            image_path = pathstr(
                directory_path,
                f"{image_size}x,pad={padding},sw={stroke_width} {kvg.kvgid}.png",
            )
            image.save(image_path)


# save_pngs(charutil.kanjis.all(), image_size=64, padding=4, stroke_width=2)
# save_pngs(charutil.kanjis.all(), image_size=16, padding=0, stroke_width=2)

In [ ]:
# !find ./output -type f -name "*.png" | wc -l